In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import sys
import urllib
from googleapiclient.discovery import build
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
import time

# Code to read in most recent monday scraped data

In [10]:
current_date = datetime.now()
days_to_monday = current_date.weekday() - 0  # 0 corresponds to Monday
most_recent_monday = current_date - timedelta(days=days_to_monday)
file_name = f"../Data/OSG_Podcast_Data{most_recent_monday.strftime('%Y-%m-%d')}.csv"
df = pd.read_csv(file_name)

In [11]:
df['Date Collected'] = pd.to_datetime(df['Date Collected'], format='%m/%d/%y', errors='coerce')
pd.set_option('display.max_colwidth', None)

# Table for Total Plays

In [12]:
most_recent_date = df["Date Collected"].max()
most_recent = df[df["Date Collected"] == most_recent_date]

In [13]:
sorted_df = most_recent.sort_values(by="Total Combined Plays", ascending=False)
sorted_df["Rank"] = range(1, len(sorted_df) + 1)
Total_plays_df = sorted_df[["Rank", "Episode Title", "Total Combined Plays", "Date"]]
Total_plays_df

,Rank,Episode Title,Total Combined Plays,Date
22,1,David Brooks: What is a Meaningful Life?,22177,03-21-2023
10,2,Dr. Rachel Naomi Remen: Can We All Be Healers?,14238,09-05-2023
19,3,You’re Not Alone in Feeling Lonely,12323,05-02-2023
26,4,Dr. Marisa G. Franco: Why Do Friendships Matter? (Part 2),11468,02-07-2023
15,5,"Meet My Moai, A Powerful Friendship Tradition",11242,06-27-2023
32,6,Bonus: Meditation for Easing Holiday Stress,11008,12-20-2022
13,7,Rainn Wilson: Walking A Spiritual Path In A Lonely World,10662,07-25-2023
14,8,Richard Reeves: Why Are Boys And Men Struggling For Connection?,10504,07-11-2023
17,9,Dr. Lisa Damour: How Can We Protect Teen Mental Health?,10271,05-30-2023
16,10,Judy Woodruff: What Does It Mean To Really Listen?,10156,06-13-2023


# Table for Top 20 youtube plays

In [15]:
sorted_df = most_recent.sort_values(by="Total YouTube Plays", ascending=False)
sorted_df["Rank"] = range(1, len(sorted_df) + 1)
Top20_YT_plays_df = sorted_df.nlargest(20, "Total YouTube Plays")[["Rank", "Episode Title", "Total YouTube Plays", "Date"]]
Top20_YT_plays_df

,Rank,Episode Title,Total YouTube Plays,Date
32,1,Bonus: Meditation for Easing Holiday Stress,10069,12-20-2022
26,2,Dr. Marisa G. Franco: Why Do Friendships Matter? (Part 2),9150,02-07-2023
22,3,David Brooks: What is a Meaningful Life?,8612,03-21-2023
27,4,Bonus: Meditation for Getting Through a Hectic Day,8535,01-31-2023
31,5,Bonus: Meditation for Combating Loneliness,7089,01-04-2023
38,6,Kal Penn: Taking the Path of Most Resistance,5909,10-04-2022
19,7,You’re Not Alone in Feeling Lonely,5621,05-02-2023
28,8,Dr. Marisa G. Franco: Why Do Friendships Matter? (Part 1),5468,01-24-2023
30,9,Maya Shankar: Why is Change So Hard?,5170,01-10-2023
42,10,Father Greg Boyle: Finding Strength Through Kindness,4303,08-09-2022


# Table for Top 20 Podcast Plays

In [16]:
sorted_df = most_recent.sort_values(by="Total Podcast Plays", ascending=False)
sorted_df["Rank"] = range(1, len(sorted_df) + 1)
Top20_pod_plays_df = sorted_df.nlargest(20, "Total Podcast Plays")[["Rank", "Episode Title", "Total Podcast Plays", "Date"]]
Top20_pod_plays_df

,Rank,Episode Title,Total Podcast Plays,Date
22,1,David Brooks: What is a Meaningful Life?,13565,03-21-2023
10,2,Dr. Rachel Naomi Remen: Can We All Be Healers?,10255,09-05-2023
13,3,Rainn Wilson: Walking A Spiritual Path In A Lonely World,8865,07-25-2023
11,4,Encore | Susan Cain: How Do We Navigate the Joy and Sorrow of Being Human?,8650,08-22-2023
17,5,Dr. Lisa Damour: How Can We Protect Teen Mental Health?,8569,05-30-2023
7,6,Baratunde Thurston: What Can Nature Teach Us About Connection?,8126,10-17-2023
14,7,Richard Reeves: Why Are Boys And Men Struggling For Connection?,8034,07-11-2023
16,8,Judy Woodruff: What Does It Mean To Really Listen?,7931,06-13-2023
15,9,"Meet My Moai, A Powerful Friendship Tradition",7877,06-27-2023
8,10,Ashley Judd & Friends: Finding Chosen Family,7603,10-03-2023


# Table for Apple Engaged Listeners

In [17]:
sorted_df = most_recent.sort_values(by="Unique Engaged Listeners (A)", ascending=False)
sorted_df["Rank"] = range(1, len(sorted_df) + 1)
Engaged_listens_df = sorted_df[["Rank", "Episode Title", "Unique Engaged Listeners (A)"]]
Engaged_listens_df

,Rank,Episode Title,Unique Engaged Listeners (A)
22,1,David Brooks: What is a Meaningful Life?,2754.0
10,2,Dr. Rachel Naomi Remen: Can We All Be Healers?,1888.0
21,3,Dr. Rangan Chatterjee: How Stressed Are We? (And What Can We Do About It?) (Part 1),1767.0
6,4,Yo-Yo Ma: How Music Heals Us (Part 1),1762.0
13,5,Rainn Wilson: Walking A Spiritual Path In A Lonely World,1618.0
19,6,You’re Not Alone in Feeling Lonely,1600.0
20,7,Dr. Rangan Chatterjee: What's at the Core of Your Happiness? (Part 2),1540.0
11,8,Encore | Susan Cain: How Do We Navigate the Joy and Sorrow of Being Human?,1480.0
16,9,Judy Woodruff: What Does It Mean To Really Listen?,1470.0
7,10,Baratunde Thurston: What Can Nature Teach Us About Connection?,1327.0


# Table for Times Played

In [18]:
sorted_df = most_recent.sort_values(by="Times Played", ascending=False)
sorted_df["Rank"] = range(1, len(sorted_df) + 1)
Times_fully_played_df = sorted_df[["Rank", "Episode Title", "Times Played"]]
Times_fully_played_df.rename(columns={'Times Played': 'Times Fully Played'}, inplace=True)
Times_fully_played_df

/var/folders/ry/gtyjy22x1fb9w3flcp8g9rlw0000gn/T/ipykernel_9075/1025491006.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Times_fully_played_df.rename(columns={'Times Played': 'Times Fully Played'}, inplace=True)


,Rank,Episode Title,Times Fully Played
22,1,David Brooks: What is a Meaningful Life?,4943.41
21,2,Dr. Rangan Chatterjee: How Stressed Are We? (And What Can We Do About It?) (Part 1),2970.74
10,3,Dr. Rachel Naomi Remen: Can We All Be Healers?,2965.72
6,4,Yo-Yo Ma: How Music Heals Us (Part 1),2648.52
20,5,Dr. Rangan Chatterjee: What's at the Core of Your Happiness? (Part 2),2530.80
19,6,You’re Not Alone in Feeling Lonely,2527.58
13,7,Rainn Wilson: Walking A Spiritual Path In A Lonely World,2382.09
11,8,Encore | Susan Cain: How Do We Navigate the Joy and Sorrow of Being Human?,2239.60
16,9,Judy Woodruff: What Does It Mean To Really Listen?,2133.33
17,10,Dr. Lisa Damour: How Can We Protect Teen Mental Health?,2068.89


# Cumulative Table

In [19]:
def calculate_platform_stats(df, latest):
    # Calculate All Platform Average Plays Per Episode
    all_platform_avg_plays = round(df['Total Combined Plays'].mean())

    # Calculate All Platform Total Plays
    all_platform_total_plays = df['Total Combined Plays'].sum()

    # Calculate All Audio Platform Plays
    all_audio_platform_plays = df['Total Podcast Plays'].sum()

    # Calculate All YouTube Plays
    all_youtube_plays = df['Total YouTube Plays'].sum()
    df['Streams (S)'] = pd.to_numeric(df['Streams (S)'], errors='coerce')
        
    # Calculate All Spotify Plays
    all_streams = df['Streams (S)'].sum()
    all_hours = df['Hours Played (S)'].sum()
    
    # Calculate All Apple Plays
    apple_engaged_listeners = df['Unique Engaged Listeners (A)'].sum()
    apple_total_time_listened = df['Total Hours Played'].sum()


    # Extract only the date part from the datetime
    latest_date = latest.date()

    # Create a new DataFrame with the calculated statistics
    stats_df = pd.DataFrame({
        'Metric': ['All Platform Average Plays Per Episode',
                   'All Platform Total Plays (Spotify, Apple, YouTube)',
                   'All Audio Platform Plays (Spotify + Apple)',
                   'All YouTube Plays', 
                   'Spotify Listener Streams (60 seconds or more)', 
                   'Spotify Consumption Hours',
                   'Apple Engaged Listeners (20 minutes or >40%)',
                   'Apple Total Time Listened (hours)'],
        latest_date: [all_platform_avg_plays, all_platform_total_plays,
                      all_audio_platform_plays, all_youtube_plays,all_streams,
                      all_hours,apple_engaged_listeners,apple_total_time_listened]
    })

    return stats_df

In [20]:
unique_dates = df["Date Collected"].unique()
unique_dates_sorted = sorted(unique_dates, reverse=True)
latest = df["Date Collected"].max()
latest1 = unique_dates_sorted[1] if len(unique_dates_sorted) > 1 else None
latest2 = unique_dates_sorted[2] if len(unique_dates_sorted) > 1 else None
latest3 = unique_dates_sorted[3] if len(unique_dates_sorted) > 1 else None
latest_df = df[df["Date Collected"] == latest]
latest_df1 = df[df["Date Collected"] == latest1]
latest_df2 = df[df["Date Collected"] == latest2]
latest_df3 = df[df["Date Collected"] == latest3]
df1 = calculate_platform_stats(latest_df, latest)
df2 = calculate_platform_stats(latest_df1, latest1)
df3 = calculate_platform_stats(latest_df2, latest2)
df4 = calculate_platform_stats(latest_df3, latest3)
concatenated_df = pd.concat([df4, df3, df2, df1], axis=1)
concatenated_df = concatenated_df.loc[:, ~concatenated_df.columns.duplicated()]
concatenated_df

/var/folders/ry/gtyjy22x1fb9w3flcp8g9rlw0000gn/T/ipykernel_9075/671813604.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Streams (S)'] = pd.to_numeric(df['Streams (S)'], errors='coerce')


,Metric,2024-01-15,2024-01-24,2024-01-29,2024-02-05
0,All Platform Average Plays Per Episode,7889.0,7962.0,7941.0,7655.0
1,"All Platform Total Plays (Spotify, Apple, YouTube)",362908.0,366243.0,373212.0,359786.0
2,All Audio Platform Plays (Spotify + Apple),217582.0,220255.0,226266.0,229427.0
3,All YouTube Plays,145326.0,145988.0,146946.0,130359.0
4,Spotify Listener Streams (60 seconds or more),47888.0,16366.0,15095.0,15673.0
5,Spotify Consumption Hours,17084.0,17447.0,17925.0,18324.0
6,Apple Engaged Listeners (20 minutes or >40%),41083.0,41645.0,42588.0,43272.0
7,Apple Total Time Listened (hours),55118.0,56012.0,57576.0,58773.0


# Cumulative Episode

In [30]:
last_eps = most_recent["#"].max() 
last_eps_df = most_recent[most_recent["#"] == last_eps]
last_eps_df

,#,Episode Title,Total Combined Plays,Total Podcast Plays,Total YouTube Plays,Duration,Times Played,Median Play Time (S),% Complete (S),Total Hours Played,Hours Played (S),Hours Played (A),Date,Unique Listeners (A),Unique Engaged Listeners (A),Plays (A),Starts (S),Streams (S),Listeners (S),Date Collected
0,46,Dr. Lisa Miller: How Does Spirituality Protect Our Mental Health?,6523,5960,563,01:07:38,1476.2,00:56:17,45,1664.0,429.0,1235,01-24-2024,1446.0,937.0,9022.0,1199.0,907,727.0,2024-02-05
